#### Initialize ####

In [1]:
function tf {aws-vault exec terraform_user -- terraform $args}

function prompt_to_confirm { 
    $user_response = Read-host "Continue? y/n or 1/0"
    if ($user_response -ne "y" -and $user_response -ne 1) {break}
}

In [2]:
$aws_region = "us-east-2"
$env:AWS_REGION=$aws_region #for aws_vault
$env:TF_VAR_aws_region=$aws_region #for terraform

In [9]:
terraform init #-upgrade

Initializing the backend...
Initializing modules...
- webserver_cluster in .terraform\modules\webserver_cluster\configurations_modules\webserver_cluster
- webserver_cluster.alb in .terraform\modules\webserver_cluster.alb\general_purpose_modules\networking\alb
- webserver_cluster.asg in .terraform\modules\webserver_cluster.asg\general_purpose_modules\asg-rolling-deploy
Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/aws v5.6.2

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


#### Workflow ####

In [10]:
terraform fmt

main.tf
outputs.tf


In [11]:
terraform validate

Success! The configuration is valid.



In [19]:
#Enable egress temporarily to install software after deployment
$env:TF_VAR_enable_egress=1

In [13]:
###      ###
### PLAN ###
###      ###

tf plan -out=tfplan
terraform show -json tfplan > plan.json
Get-Date

data.aws_ami.ami: Reading...
module.webserver_cluster.module.asg.data.aws_ec2_instance_type.instance: Reading...
module.webserver_cluster.data.aws_vpc.existing_vpc: Reading...
module.webserver_cluster.module.asg.data.aws_ec2_instance_type.instance: Read complete after 1s [id=t2.micro]
data.aws_ami.ami: Read complete after 1s [id=ami-09da212cf18033880]
module.webserver_cluster.data.aws_vpc.existing_vpc: Read complete after 2s [id=vpc-0a4860611e96d1d85]
module.webserver_cluster.data.aws_subnets.vpc_subnets: Reading...
module.webserver_cluster.data.aws_subnets.vpc_subnets: Read complete after 0s [id=us-east-2]
module.webserver_cluster.data.aws_subnet.subnets["subnet-0e6c5590894f5f7cd"]: Reading...
module.webserver_cluster.data.aws_subnet.subnets["subnet-0075ecc8eac5c3cdd"]: Reading...
module.webserver_cluster.data.aws_subnet.subnets["subnet-0c13b6812ab6e3f89"]: Reading...
module.webserver_cluster.data.aws_subnet.subnets["subnet-07a06d61b264fbaeb"]: Reading...
module.webserver_cluster.data

In [14]:
#Output planned resource addresses and their names in the cloud

$plan = Get-Content -Raw -Path "plan.json" | ConvertFrom-Json

$resourcesToDeploy = $plan.resource_changes | Where-Object { 
    $_.change.actions -contains "create" -or $_.change.actions -contains "update" 
} | ForEach-Object { 
    "Type: " + $_.type
    "TF name: " + $_.name
    "Cloud name: " + $_.change.after.name
    "Name tag: " + $_.change.after.tags_all.name
    ""
}

$resourcesToDeploy
Get-Date

Type: aws_lb_listener_rule
TF name: asg
Cloud name: 
Name tag: 

Type: aws_lb_target_group
TF name: asg
Cloud name: hello-world-prod
Name tag: 

Type: aws_lb
TF name: example
Cloud name: hello-world-prod
Name tag: 

Type: aws_lb_listener
TF name: http
Cloud name: 
Name tag: 

Type: aws_security_group
TF name: alb
Cloud name: hello-world-prod
Name tag: 

Type: aws_security_group_rule
TF name: allow_all_outbound
Cloud name: 
Name tag: 

Type: aws_security_group_rule
TF name: allow_http_inbound
Cloud name: 
Name tag: 

Type: aws_autoscaling_group
TF name: example
Cloud name: webserver_cluster-prod
Name tag: 

Type: aws_cloudwatch_metric_alarm
TF name: high_cpu_utilization
Cloud name: 
Name tag: 

Type: aws_cloudwatch_metric_alarm
TF name: low_cpu_credit_balance
Cloud name: 
Name tag: 

Type: aws_launch_configuration
TF name: example
Cloud name: 
Name tag: 

Type: aws_security_group
TF name: instance
Cloud name: webserver_cluster-prod-instance
Name tag: 

Type: aws_security_group_rule
TF n

In [21]:
###       ###
### APPLY ###
###       ###

try {prompt_to_confirm} catch {break}
tf apply -auto-approve
Get-Date

module.webserver_cluster.data.aws_vpc.existing_vpc: Reading...
module.webserver_cluster.module.asg.data.aws_ec2_instance_type.instance: Reading...
data.aws_ami.ami: Reading...
data.aws_ami.ami: Read complete after 2s [id=ami-09da212cf18033880]
module.webserver_cluster.module.asg.data.aws_ec2_instance_type.instance: Read complete after 3s [id=t2.micro]
module.webserver_cluster.data.aws_vpc.existing_vpc: Read complete after 3s [id=vpc-0a4860611e96d1d85]
module.webserver_cluster.data.aws_subnets.vpc_subnets: Reading...
module.webserver_cluster.data.aws_subnets.vpc_subnets: Read complete after 1s [id=us-east-2]
module.webserver_cluster.data.aws_subnet.subnets["subnet-05c781f49cb7a7b5e"]: Reading...
module.webserver_cluster.data.aws_subnet.subnets["subnet-0c13b6812ab6e3f89"]: Reading...
module.webserver_cluster.data.aws_subnet.subnets["subnet-07373a76cf5740bbb"]: Reading...
module.webserver_cluster.data.aws_subnet.subnets["subnet-0075ecc8eac5c3cdd"]: Reading...
module.webserver_cluster.data

Error: Command failed: SubmitCode: ###       ### ...

In [24]:
#Disable egress after deployment
$env:TF_VAR_enable_egress=0

In [25]:
#Deploy again to remove egress security rule

try {prompt_to_confirm} catch {break}
tf apply -auto-approve
Get-Date

module.webserver_cluster.module.asg.data.aws_ec2_instance_type.instance: Reading...
data.aws_ami.ami: Reading...
module.webserver_cluster.data.aws_vpc.existing_vpc: Reading...
module.webserver_cluster.module.asg.data.aws_ec2_instance_type.instance: Read complete after 1s [id=t2.micro]
data.aws_ami.ami: Read complete after 1s [id=ami-09da212cf18033880]
module.webserver_cluster.data.aws_vpc.existing_vpc: Read complete after 2s [id=vpc-0a4860611e96d1d85]
module.webserver_cluster.data.aws_subnets.vpc_subnets: Reading...
module.webserver_cluster.module.alb.aws_security_group.alb: Refreshing state... [id=sg-029d73a8866caddcb]
module.webserver_cluster.module.asg.aws_security_group.instance: Refreshing state... [id=sg-048972fc852019424]
module.webserver_cluster.aws_lb_target_group.asg: Refreshing state... [id=arn:aws:elasticloadbalancing:us-east-2:588738573172:targetgroup/hello-world-prod/6a50e9dd6a0505a5]
module.webserver_cluster.data.aws_subnets.vpc_subnets: Read complete after 0s [id=us-eas

In [1]:
$public_ip = terraform output -raw public_ip
$http_port = terraform output -raw LB_http_port

In [2]:
(Invoke-WebRequest -Uri ("http://" + $public_ip + ":$LB_http_port")).Content

https://github.com/aksenov-fx



In [ ]:
###         ###
### DESTROY ###
###         ###

try {prompt_to_confirm} catch {break}
tf destroy -auto-approve
Get-Date

#### Info ####

In [3]:
terraform version

Terraform v1.9.5
on windows_amd64
+ provider registry.terraform.io/hashicorp/aws v5.6.2

Your version of Terraform is out of date! The latest version
is 1.9.7. You can update by downloading from https://www.terraform.io/downloads.html


In [4]:
#Root module configuration
cat .\main.tf

terraform {
  required_version = ">= 1.9.5, < 2.0"

  required_providers {
    aws = {
      source = "hashicorp/aws"
    }
  }
}

provider "aws" {
  region = var.aws_region
}

# --- --- --- --- --- --- --- --- --- --- #

locals {
  vpc_name = "TF-VPC1"
}

# --- --- --- --- --- --- --- --- --- --- #

# Get ami_id
data "aws_ami" "ami" {
  most_recent = true
  owners      = ["137112412989"] #Amazon
  filter {
    name   = "name"
    values = ["al2023-ami-2023.5*"] #Amazon_Linux_2023
  }
}

# --- --- --- --- --- --- --- --- --- --- #

module "webserver_cluster" {

  source = "git::https://github.com/aksenov-fx/terraform-modules.git//configurations_modules/webserver_cluster"
  #source = "../../configurations_modules/webserv_cluster"

  user_data = base64encode(templatefile("${path.module}/user-data.sh", {
    http_port   = var.http_port
    server_text = var.server_text
  }))

  environment = var.environment

  instance_type      = "t2.micro"
  min_size           = 2
  max_size           =

In [5]:
($resources = terraform state list) | Out-Host

data.aws_ami.ami
module.webserver_cluster.data.aws_subnet.subnets["subnet-0075ecc8eac5c3cdd"]
module.webserver_cluster.data.aws_subnet.subnets["subnet-05c781f49cb7a7b5e"]
module.webserver_cluster.data.aws_subnet.subnets["subnet-07373a76cf5740bbb"]
module.webserver_cluster.data.aws_subnet.subnets["subnet-07a06d61b264fbaeb"]
module.webserver_cluster.data.aws_subnet.subnets["subnet-0c13b6812ab6e3f89"]
module.webserver_cluster.data.aws_subnet.subnets["subnet-0e6c5590894f5f7cd"]
module.webserver_cluster.data.aws_subnets.vpc_subnets
module.webserver_cluster.data.aws_vpc.existing_vpc
module.webserver_cluster.aws_lb_listener_rule.asg
module.webserver_cluster.aws_lb_target_group.asg
module.webserver_cluster.module.alb.aws_lb.example
module.webserver_cluster.module.alb.aws_lb_listener.http
module.webserver_cluster.module.alb.aws_security_group.alb
module.webserver_cluster.module.alb.aws_security_group_rule.allow_all_outbound
module.webserver_cluster.module.alb.aws_security_group_rule.allow_http_

In [ ]:
foreach ($resource in $resources) { terraform state show $resource }

In [6]:
terraform output

LB_http_port = 80
public_ip = "hello-world-prod-1828154032.us-east-2.elb.amazonaws.com"


#### Troubleshooting ####

In [ ]:
#$env:TF_LOG = "TRACE"
#$env:TF_LOG_PATH = "log.txt"
#$env:TF_LOG = ""

In [ ]:
tf apply -refresh-only -auto-approve